In [ ]:
from __future__ import print_function, division

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os, sys

In [ ]:
import copy
import time
import json
import tensorflow as tf
import keras
from keras.layers import Input, Lambda, Concatenate
from keras import Model
import keras.backend as K
from keras.applications.vgg16 import preprocess_input

In [ ]:
import ray
ray.init()

In [ ]:
print(keras.__version__, tf.__version__)

In [ ]:
from models import QATM, MyNormLayer
from utils import compute_score, locate_bbox

In [ ]:
## Load data
file_dir = '/home/yonif/SimulAI/SimulationsBW3'
sub_directories = [os.path.join(file_dir, i) for i in os.listdir(file_dir)] 
img_path = []
for directory in sub_directories:
    img_path.extend([os.path.join(directory, i) for i in os.listdir(directory) if '.png' in i ])
    
def read_gt( file_path ):
    with open( file_path ) as IN:
        x, y, w, h = [ eval(i) for i in IN.readline().strip().split(',')]
    return x, y, w, h
   

In [ ]:
def create_model( featex, alpha = 1. ):
    T = Input( (None, None, 3), name='template_input' )
    I = Input( (None, None, 3), name='image_input' )
    T_feat = featex(T)
    I_feat = featex(I)
    I_feat, T_feat = MyNormLayer( name='norm_layer' )( [I_feat, T_feat] )
    dist = Lambda( lambda x: tf.einsum( "xabc,xdec->xabde", K.l2_normalize(x[0], axis=-1), K.l2_normalize(x[1], axis=-1) ) , name="cosine_dist")([ I_feat, T_feat ])
    conf_map = QATM(alpha, name='qatm')( dist )
    return Model( [T, I], [conf_map], name='QATM_model')

def model_eval( featex, alpha=1., backup=None ):
    '''
    Have a backup featex in case image is too big.
    '''
    model = create_model( featex , alpha=alpha)
    if backup is not None:
        model_bkup = create_model( backup , alpha=alpha)
    else:
        model_bkup = model
    gt_list, gray_list, score_list = [], [], []
    
    num_samples = len(img_path) // 2
    bar = progressbar.ProgressBar(max_value=num_samples)
    for idx in range(num_samples):
        bar.update(idx + 1)
        template_raw = cv2.imread( img_path[2*idx] )[...,::-1]
        template_bbox = read_gt( gt[2*idx] )
        x, y, w, h = [int(round(t)) for t in template_bbox]
        template = template_raw[y:y+h, x:x+w]
        image = cv2.imread( img_path[2*idx+1] )[...,::-1]
        image_gt = read_gt( gt[2*idx+1] )
        x_gt, y_gt, w_gt, h_gt = [int(round(t)) for t in image_gt]
        
        # process images
        template_ = np.expand_dims(preprocess_input( template ), axis=0)
        image_ = np.expand_dims(preprocess_input( image ) , axis=0)
        if w*h <= 4000:
            val = model.predict( [template_, image_] )
        else:
            # used when image is too big
            val = model_bkup.predict( [template_, image_] )
        
        # compute geometry mean on score map
        val = np.log( val )
        gray = val[0,:,:,0]
        gray = cv2.resize( gray, (image.shape[1], image.shape[0]) )
        score = compute_score( gray, w_gt, h_gt )
        score[score>-1e-7] = -np.inf
        
        gt_list.append( image_gt )
        gray_list.append( gray )
        score_list.append( score )
    return score_list, gt_list, gray_list


In [ ]:
vgg19 = keras.applications.vgg19.VGG19( include_top = False, weights = 'imagenet' )
   

In [ ]:
@ray.remote
def run_template_to_db(experiment_template_path):  
    #creating the model
    ####################################################
     # resize conv3_4 to conv1_2
    vgg19 = keras.applications.vgg19.VGG19( include_top = False, weights = 'imagenet' )
    input_ = vgg19.input
    conv1_2 = vgg19.get_layer('block1_conv2').output
    conv3_4 = vgg19.get_layer('block3_conv4').output
    conv3_4 = Lambda( lambda x: tf.image.resize_bilinear( x[0], size=(tf.shape(x[1])[1], tf.shape(x[1])[2]), align_corners=True), name='resized_image' )( [conv3_4, conv1_2] )
    concat = Concatenate()( [conv1_2, conv3_4] )
    featex = Model( [input_], [concat], name='featex' )
    # resize conv1_2 to conv3_4, used when image size is too big
    input_ = vgg19.input
    conv1_2 = vgg19.get_layer('block1_conv2').output
    conv3_4 = vgg19.get_layer('block3_conv4').output
    conv1_2 = Lambda( lambda x: tf.image.resize_bilinear( x[1], size=(tf.shape(x[0])[1], tf.shape(x[0])[2]), align_corners=True), name='resized_image' )( [conv3_4, conv1_2] )
    concat = Concatenate()( [conv1_2, conv3_4] )
    featex2 = Model( [input_], [concat], name='featex2' )
    model = create_model( featex , alpha=25)
    model_bkup = create_model( featex2 , alpha=25)
    
    ####################################################


    template_path = experiment_template_path ################################# ASSUME IMAGES COME THRESHOLDED!
    # load image and ground truth
    template_raw = cv2.imread(template_path)[...,::-1]
    x_t = 0
    y_t = 0
    w_t =  template_raw.shape[1]-1
    h_t =  template_raw.shape[0]-1
    template_plot = cv2.rectangle( template_raw.copy(), (x_t, y_t), (x_t+w_t, y_t+h_t), (0, 255,0), 1 )
    template = template_raw[y_t:y_t+h_t, x_t:x_t+w_t]
    
    #we noticed that some images crash the running, so we created one json to save the data we succeeded to examine, and other json to save the paths of the bad images   
    file_json_good = open("/home/yonif/SimulAI/QATM/QATM_GOOD_JSON/" + os.path.basename(experiment_template_path)[:-4] + ".json", "w")
    file_json_bad = open("/home/yonif/SimulAI/QATM/QATM_BAD_JSON/" + os.path.basename(experiment_template_path)[:-4] + ".json", "w")
    
    count_good = 0 
    for i in range(len(img_path)):
        if i%100==0 and i>0:
            t = time.localtime()
            current_time = time.strftime("%H:%M:%S", t)
            print (experiment_template_path, count_good, i , current_time)
        image = cv2.imread( img_path[i] )[...,::-1]
        template_ = np.expand_dims(preprocess_input( template ), axis=0)
        image_ = np.expand_dims(preprocess_input( image ) , axis=0)
        try:     
            if w_t*h_t <= 4000:             
                val = model.predict( [template_, image_] )

            else:                
                val = model_bkup.predict( [template_, image_] )
            # compute geometry average on score map
            val = np.log( val )
            gray = val[0,:,:,0]
            gray = cv2.resize( gray, (image.shape[1], image.shape[0]) )
            score = compute_score( gray, w_t, h_t)    
            score[score>-1e-7] = score.min()
            score = np.exp(score / (h_t*w_t)) # reverse number range back after computing geometry average
            max_score, x, y, w, h = locate_bbox(score, w_t, h_t)
                                           
            if i == 0 :
                file_json_good.write("[" + str(max_score) + ",\"" + img_path[i] + "\"," + "(" + str(int(x)) + "," + str(int(y)) + "," + str(int(h)) + "," + str(int(w))  +")," + str(i) + "]")
            else:
                file_json_good.write(",[" + str(max_score) + ",\"" + img_path[i] + "\"," + "(" + str(int(x)) + "," + str(int(y)) + "," + str(int(h)) + "," + str(int(w))  +")," + str(i) + "]")

            count_good= count_good+1  
            
        except:          
            if i == 0 :            
                file_json_bad.write(json.dumps(img_path[i]))
            else:
                file_json_bad.write("," + json.dumps(img_path[i])) 

                

In [ ]:
from functools import singledispatch

file_dir = '/home/yonif/SimulAI/SimulationsBW3'
sub_directories = [os.path.join(file_dir, i) for i in os.listdir(file_dir)] 
img_path = []
for directory in sub_directories:
    img_path.extend([os.path.join(directory, i) for i in os.listdir(directory) if '.png' in i ])
 

@singledispatch
def to_serializable(val):
    #Used by default
    return str(val)

@to_serializable.register(np.float32)
def ts_float32(val):
    #Used if *val* is an instance of numpy.float32.
    return np.float64(val)


file_dir = '/home/yonif/SimulAI/QATM/Corrected_Templates'
os.makedirs("/home/yonif/SimulAI/QATM/QATM_GOOD_JSON")
os.makedirs("/home/yonif/SimulAI/QATM/QATM_BAD_JSON")
tasks = list()
template_experiment_paths = [os.path.join(file_dir, i) for i in os.listdir(file_dir)]
for path in template_experiment_paths: 
    print("start work on template --- " , os.path.basename(path))
    tasks.append(run_template_to_db.remote(path))  

for task in tasks:
    ray.get(task)

print("done!!")